#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [8]:
import pandas as pd
import pymongo

In [9]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [10]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [11]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Nettoyer les données

In [12]:
#Création de la nouvelle dataframe à partir du fichier csv
new_df_ks = df_ks.copy()

#Changement de l'id
new_df_ks['ID'] = pd.Series(range(1, (len(new_df_ks['ID'])+1)))

#La ligne associé à l'ID 66142, donc index 66141 est faussé car les valeurs ne sont pas dans la bonne colonne rendant la ligne inutilisable et rendant certaines manipulations compliquées. Nous enlevons donc cette valeur
new_df_ks = new_df_ks.drop([66141])

#Modification du format de date
new_df_ks['launched'] = pd.to_datetime(new_df_ks['launched'], format='%Y-%m-%d %H:%M:%S')
new_df_ks['deadline'] = pd.to_datetime(new_df_ks['deadline'], format='%Y-%m-%d')


new_df_ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,1,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,2,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,3,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,4,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,5,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


### Importer les données

In [14]:
collection.delete_many({})
collection.insert_many(new_df_ks.to_dict(orient='records'))

TypeError: 'Collection' object is not callable. If you meant to call the 'insert_many' method on a 'Collection' object it is failing because no such method exists.

## Question 1  

In [ ]:
#On récupère les 5 projets ayant la plus grande valeur dans le colonne 'pledged'
cur = collection.find({}, {'ID':1, 'name':1, 'pledged':1}).sort([('pledged',-1)])
list(cur)[0:5]

## Question 2

In [8]:
#On compte le nombre de projet ayant la valeur 'successful' dans la colonne 'state'
cur = collection.aggregate([
    {
        '$match': { 'state': 'successful'}
    },
    {
        '$count':'Nombre de projet ayant atteint leur but'
    }])
list(cur)

[{'Nombre de projet ayant atteint leur but': 53040}]

## Question 3

In [9]:
#On compte le nombre de projets pour chaque catégorie
cur = collection.aggregate([{'$group' : {'_id' : '$category', "nombre de projets": {"$sum" : 1}}}])
list(cur)

[{'_id': 'Food Trucks', 'nombre de projets': 687},
 {'_id': 'Quilts', 'nombre de projets': 32},
 {'_id': 'Horror', 'nombre de projets': 525},
 {'_id': 'Classical Music', 'nombre de projets': 1064},
 {'_id': 'Printing', 'nombre de projets': 83},
 {'_id': 'Pop', 'nombre de projets': 1302},
 {'_id': 'Web', 'nombre de projets': 2017},
 {'_id': 'Events', 'nombre de projets': 322},
 {'_id': 'Experimental', 'nombre de projets': 357},
 {'_id': 'Digital Art', 'nombre de projets': 524},
 {'_id': 'Art', 'nombre de projets': 3358},
 {'_id': 'Product Design', 'nombre de projets': 8886},
 {'_id': 'Farms', 'nombre de projets': 482},
 {'_id': 'Faith', 'nombre de projets': 439},
 {'_id': 'Pet Fashion', 'nombre de projets': 51},
 {'_id': 'Candles', 'nombre de projets': 168},
 {'_id': 'Hardware', 'nombre de projets': 1431},
 {'_id': 'Musical', 'nombre de projets': 367},
 {'_id': 'Apparel', 'nombre de projets': 2827},
 {'_id': 'Webcomics', 'nombre de projets': 259},
 {'_id': 'Fashion', 'nombre de projets'

## Question 4

In [10]:
#On compte le nombre de projets ayant la valeur 'FR' dans la colonne 'country' et dont la date d'instanciation est avant 2016
cur = collection.aggregate([
    {
        '$match': 
        { 
            'country': 'FR',
            'launched': {'$lt': pd.to_datetime("2016-01-01")}
        }
    },
    {
        '$count':'Nombre de projets français instanciés avant 2016'
    }])
list(cur)

[{'Nombre de projet français instanciés avant 2016': 330}]

## Question 5

In [11]:
#On compte le nombre de projets ayant la valeur 'US' dans la colonne 'country', et une valeur strictement supérieur à 200 000 dans la colonne 'goal'
cur = collection.aggregate([
    {
        '$match': 
        { 
            'country': 'US',
            'goal': {'$gt': 200000}
        }
    },
    {
        '$count':"Nombre de projets américains ayant demandés plus de 200 000 dollars"
    }])
list(cur)



[{'Nombre de projets américains ayant demandé plus de 200 000 dollars': 1026}]

## Question 6 

In [1]:
#On compte le nombre de projets ayant la string "Sport" dans leur nom
cur = collection.aggregate([
    {
        '$match' : {'name' : {'$regex' : 'sport'}}
    },
    {
        '$count':"Nombre de projets contenant le mot 'Sport' dans le nom"
    }
])
list(cur)

NameError: name 'collection' is not defined